In [1]:

import numpy as np
import scipy
from scipy.misc import imread, imsave, imresize
import skimage
from skimage import data
from matplotlib import pyplot as plt
from skimage.util.shape import view_as_blocks
from skimage.transform import rescale, resize, downscale_local_mean
from skimage import feature
from skimage.transform import warp, AffineTransform
from skimage import io
%matplotlib inline
from PIL import Image
import math
from skimage.feature import hog

#************************************Function SECTION************************************   
def SUB_B(image, (M,N)):
    sub_Block = view_as_blocks(image, block_shape=(M,N))
    return sub_Block
#************************************Function SECTION************************************   

def myGrad_X(sub_Block,(max_row,max_col)):
    grad_sub_Block=np.empty([max_col,max_row]) 
    for row in range(0,max_row):
        for col in range(1,max_col-1):
                grad_sub_Block[row,col]=sub_Block[row,col-1]-sub_Block[row,col+1]  
        grad_sub_Block[row,0]= sub_Block[row,1]-sub_Block[row,0]
        grad_sub_Block[row,max_col-1]= sub_Block[row,max_col-1]-sub_Block[row,max_col-2]            
    return  grad_sub_Block   
# *********************************************************************************
def myGrad_Y(sub_Block,(max_row,max_col)):
    grad_sub_Block=np.empty([max_col,max_row]) 
    for col in range(0,max_col):
        for row in range(1,max_row-1):
            grad_sub_Block[col,row]=(sub_Block[row-1,col]-sub_Block[row+1,col])/2.0   
        grad_sub_Block[col,0]= sub_Block[1,col]-sub_Block[0,col]
        grad_sub_Block[col,max_row-1]= sub_Block[max_row-1,col]-sub_Block[max_row-2,col]
    return  grad_sub_Block   

# *********************************************************************************
def Concat_images(img_block,width,Height):
    return np.reshape(img_block,(width,Height))
# *********************************************************************************
def Preprocess(img):
    img =imread(img)
    img_gray=skimage.color.rgb2gray(img)
#     print 'shape : ', img_gray.shape  
    resized_img=skimage.transform.resize(img_gray,(256,256))
#     print 'resized_img : ', resized_img.shape
#     plt.imshow(resized_img, cmap='gray')
#     plt.show()
    return resized_img
# *********************************************************************************
def magnitude(iGx,iGy):
    iGx2 = np.power(iGx,2)
    iGy2 =  np.power(iGy,2) 
    m= np.sqrt(np.add(iGx2,iGy2))
    return m

# *********************************************************************************
def merge(mag_img,deg):   
    m = np.empty(mag_img.shape,dtype=object)
    for i in range (0,mag_img.shape[0]): 
        for j in range (0, mag_img.shape[1]):
            m[i,j]= (mag_img[i,j] , deg[i,j])
    return m

# *********************************************************************************

def Proc_Block(Sub_Blocks_Theta,Sub_Blocks_deg):

    
    Theta_cells=SUB_B(Sub_Blocks_Theta, (8,8))
    Mag_cells=SUB_B(Sub_Blocks_deg, (8,8))
    Histogram_Blocks= np.empty([0])
    for i in range(len(Theta_cells)):
        for j in  range(len(Theta_cells)):
            Histogram=ComputeCellHistogram(Theta_cells[i,j],Mag_cells[i,j])
            Histogram_Blocks = np.concatenate((Histogram_Blocks,Histogram),axis=0)   
            

    norm=np.linalg.norm(Histogram_Blocks)

    if(norm == 0):
        return Histogram_Blocks
    
    Block_descriptor= Histogram_Blocks/norm
    
    return Block_descriptor
# *********************************************************************************
def ComputeCellHistogram(Theta_cells,Mag_cells):
    deg_cells = np.rad2deg(Theta_cells).flatten()
    Hist,bin_edges=np.histogram(deg_cells,bins=9, weights=Mag_cells.flatten(), range= (-180,180))
        
    return Hist 

In [2]:

def simple_HOG(img):            
    row_index=16
    col_index=16
    plot_nr=1
    imGx=myGrad_X(img, (256,256))
    # plt.imshow(imGx, cmap='gray')
    # plt.show()
    imGy=myGrad_Y(img, (256,256))
    imGy=np.transpose(imGy)
    # plt.imshow(imGy, cmap='gray')
    # plt.show()
    mag_img=magnitude(imGx,imGy)
        
    # plt.imshow(mag_img, cmap='gray')
    # plt.show()
    Theta =np.arctan2(imGy,imGx)     
    deg = np.rad2deg(Theta)    
        
    Sub_Blocks_Theta=SUB_B(Theta, (16,16))
    Sub_Blocks_mag  =SUB_B(mag_img, (16,16))
    # # print Sub_Blocks , Sub_Blocks.shape
    hog_descriptor=np.empty([0])
    for i in range(len(Sub_Blocks_Theta)):
        for j in range(len(Sub_Blocks_Theta)):
            Block_descriptor=Proc_Block(Sub_Blocks_Theta[i,j],Sub_Blocks_mag[i,j])            
            hog_descriptor= np.concatenate((hog_descriptor,Block_descriptor),axis=0)      
            
            
            
    return hog_descriptor

In [3]:
# img='lenna.png'
# img=Preprocess(img) 
# hog_descriptor=simple_HOG(img)
# print 'hog_descriptor : \n' ,  hog_descriptor 
# print 'hog_descriptor size' , hog_descriptor.shape


In [4]:
import os 

class KNNClassifier:
    
    def __EuclideanDist(self,featureX):
        
        self.nearest_neighbour = None
        self.min_distance = None
        
        for (featureY,category) in self.train_data:  

                self.distance= np.linalg.norm(featureX-featureY)
#                 print self.distance
  
                if self.min_distance == None or self.distance < self.min_distance:
                    self.min_distance = self.distance
#                     print self.min_distance
                    self.nearest_neighbour = (feature,category)
            
        if self.nearest_neighbour is not None:
#             print self.min_distance
            return self.nearest_neighbour[1]
       
        return None
    
    def classify(self,Feature_Img,classificationMethod="EuclideanDist"):
        if classificationMethod == "EuclideanDist":
            return self.__EuclideanDist(Feature_Img)
            
        return None   
    
    def __init__(self,train_Imagelist):
        self.train_data=train_Imagelist
        

def getAllImages(path):
    Imagelist=[]
    for Parent_root,Parent_dirs, Parent_files in os.walk(path):
#         print dirs
#         print root
        for dir in Parent_dirs:
            sub_path=os.path.join(Parent_root,dir)
#         print sub_path
            for root,dirs, files in os.walk(sub_path):
                    for file in files:
                
                        if(file != '.DS_Store'):
                            sub_path_img=os.path.join(sub_path,file)
    #                         print sub_path_img
                            Imagelist= Imagelist + [(sub_path_img,dir)]
       
    return Imagelist

# print 'Imagelist : \n ',Imagelist

def extractFeatures(Imagelist):
    FeaturesList=[]
    for i,(img_path,category) in enumerate(Imagelist):
        img=Preprocess(img_path)   
        #hog_descriptor=simple_HOG(img)
        
        d = hog(img, orientations=8, pixels_per_cell=(8, 8), cells_per_block=(2, 2))

        FeaturesList=FeaturesList + [(d,category)]
        print("Feature " + str(i) + " / " + str(len(Imagelist)) + " extracted.")
    return FeaturesList

path='/Users/dsoellinger/Downloads/scene_categories' 
Imagelist=getAllImages(path)
FeaturesList=extractFeatures(Imagelist)


/usr/local/lib/python2.7/site-packages/skimage/transform/_warps.py:84: UserWarning: The default mode, 'constant', will be changed to 'reflect' in skimage 0.15.
  warn("The default mode, 'constant', will be changed to 'reflect' in "
/usr/local/lib/python2.7/site-packages/skimage/feature/_hog.py:119: skimage_deprecation: Default value of `block_norm`==`L1` is deprecated and will be changed to `L2-Hys` in v0.15
  'be changed to `L2-Hys` in v0.15', skimage_deprecation)


Feature 0 / 672 extracted.
Feature 1 / 672 extracted.
Feature 2 / 672 extracted.
Feature 3 / 672 extracted.
Feature 4 / 672 extracted.
Feature 5 / 672 extracted.
Feature 6 / 672 extracted.
Feature 7 / 672 extracted.
Feature 8 / 672 extracted.
Feature 9 / 672 extracted.
Feature 10 / 672 extracted.
Feature 11 / 672 extracted.
Feature 12 / 672 extracted.
Feature 13 / 672 extracted.
Feature 14 / 672 extracted.
Feature 15 / 672 extracted.
Feature 16 / 672 extracted.
Feature 17 / 672 extracted.
Feature 18 / 672 extracted.
Feature 19 / 672 extracted.
Feature 20 / 672 extracted.
Feature 21 / 672 extracted.
Feature 22 / 672 extracted.
Feature 23 / 672 extracted.
Feature 24 / 672 extracted.
Feature 25 / 672 extracted.
Feature 26 / 672 extracted.
Feature 27 / 672 extracted.
Feature 28 / 672 extracted.
Feature 29 / 672 extracted.
Feature 30 / 672 extracted.
Feature 31 / 672 extracted.
Feature 32 / 672 extracted.
Feature 33 / 672 extracted.
Feature 34 / 672 extracted.
Feature 35 / 672 extracted.
Fe

Feature 292 / 672 extracted.
Feature 293 / 672 extracted.
Feature 294 / 672 extracted.
Feature 295 / 672 extracted.
Feature 296 / 672 extracted.
Feature 297 / 672 extracted.
Feature 298 / 672 extracted.
Feature 299 / 672 extracted.
Feature 300 / 672 extracted.
Feature 301 / 672 extracted.
Feature 302 / 672 extracted.
Feature 303 / 672 extracted.
Feature 304 / 672 extracted.
Feature 305 / 672 extracted.
Feature 306 / 672 extracted.
Feature 307 / 672 extracted.
Feature 308 / 672 extracted.
Feature 309 / 672 extracted.
Feature 310 / 672 extracted.
Feature 311 / 672 extracted.
Feature 312 / 672 extracted.
Feature 313 / 672 extracted.
Feature 314 / 672 extracted.
Feature 315 / 672 extracted.
Feature 316 / 672 extracted.
Feature 317 / 672 extracted.
Feature 318 / 672 extracted.
Feature 319 / 672 extracted.
Feature 320 / 672 extracted.
Feature 321 / 672 extracted.
Feature 322 / 672 extracted.
Feature 323 / 672 extracted.
Feature 324 / 672 extracted.
Feature 325 / 672 extracted.
Feature 326 / 

Feature 577 / 672 extracted.
Feature 578 / 672 extracted.
Feature 579 / 672 extracted.
Feature 580 / 672 extracted.
Feature 581 / 672 extracted.
Feature 582 / 672 extracted.
Feature 583 / 672 extracted.
Feature 584 / 672 extracted.
Feature 585 / 672 extracted.
Feature 586 / 672 extracted.
Feature 587 / 672 extracted.
Feature 588 / 672 extracted.
Feature 589 / 672 extracted.
Feature 590 / 672 extracted.
Feature 591 / 672 extracted.
Feature 592 / 672 extracted.
Feature 593 / 672 extracted.
Feature 594 / 672 extracted.
Feature 595 / 672 extracted.
Feature 596 / 672 extracted.
Feature 597 / 672 extracted.
Feature 598 / 672 extracted.
Feature 599 / 672 extracted.
Feature 600 / 672 extracted.
Feature 601 / 672 extracted.
Feature 602 / 672 extracted.
Feature 603 / 672 extracted.
Feature 604 / 672 extracted.
Feature 605 / 672 extracted.
Feature 606 / 672 extracted.
Feature 607 / 672 extracted.
Feature 608 / 672 extracted.
Feature 609 / 672 extracted.
Feature 610 / 672 extracted.
Feature 611 / 

In [ ]:
#numCorrectClassifications = 0
#totalCorrectClassifications = 0

#for i,feature in enumerate(FeaturesList):
    
    # Initalize classifier without feature
 #   tmpFeatureList = list(FeaturesList)
  #  del tmpFeatureList[i]

   # objImgClassifier=KNNClassifier(tmpFeatureList) 

    #predictedCategory = objImgClassifier.classify(feature[0])

    #if(predictedCategory == feature[1]):
     #   numCorrectClassifications = numCorrectClassifications + 1
    
   # totalCorrectClassifications = totalCorrectClassifications + 1
    
#classificationRate = (numCorrectClassifications / float(totalCorrectClassifications)) * 100 

#print("Classification Rate: " + str(classificationRate))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier

numCorrectClassifications = 0
totalCorrectClassifications = 0

for i,feature in enumerate(FeaturesList):
    
    # Initalize classifier without feature
    tmpFeatureList = list(FeaturesList)
    del tmpFeatureList[i]
    
    knn = KNeighborsClassifier(1)
    
    
    newFeatureList = []
    newCategoryList = []
    for element in tmpFeatureList:
        newFeatureList = newFeatureList + [element[0]]
        newCategoryList = newCategoryList + [element[1]]

    knn.fit(newFeatureList,newCategoryList)

    predictedCategory = knn.predict([feature[0]])
    print("Predicted: " + predictedCategory[0])
    print("Real: " + feature[1])
    if(predictedCategory[0] == feature[1]):
        numCorrectClassifications += 1
    totalCorrectClassifications += 1

classificationRate = (numCorrectClassifications / float(totalCorrectClassifications)) * 100 

print("Classification Rate: " + str(classificationRate))


Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest
Real: bedroom
Predicted: MITforest